In [1]:
import sys
import os
import rdkit
import random as rand
import numpy as np
#import nglview as nv
from openbabel import openbabel as ob
from pyquaternion import Quaternion
from rdkit import Chem

In [2]:
def xyz2RDmol(xyzFile): #for some reason, RDkit won't show an RDmol it converts from an .xyz file, so I'm using this.
    convert = ob.OBConversion()
    convert.SetInAndOutFormats("xyz", "mol")
    mol = ob.OBMol()
    convert.ReadFile(mol, xyzFile) 
    convert.WriteFile(mol, xyzFile[:-4] + ".mol")
    mol = Chem.MolFromMolFile(xyzFile[:-4] + ".mol", sanitize = False, removeHs = False)
    return mol

Gets specified adsorbate groups in a given .xyz file. Takes SMILES/SMARTS input or another .xyz file as a reference for what to find.

In [3]:
dmaSMARTS = '*-[O]-[Al](C([H])([H])([H]))(C([H])([H])([H]))'
CRABxyz = 'O-CRAB.xyz'

def getCoordsAndGrps(coordFile, SMARTS = None, molfromXYZ = None):
    
    # read in xyz file, convert coords to float #
    with open(coordFile, 'r') as file:
        coords = [coord.split() for coord in file.readlines()[2:]]
    for i in range(len(coords)):
        for j in range(1, 4):
            coords[i][j] = float(coords[i][j])
    ###
    
    # get atom indices of all adsorbate groups in structure #
    mol = xyz2RDmol(coordFile)
    if SMARTS:
        substruct = Chem.MolFromSmarts(SMARTS)
        inds = [ind[1::] for ind in mol.GetSubstructMatches(substruct)]
    elif molfromXYZ:
        substruct = xyz2RDmol(molfromXYZ)
        inds = [ind for ind in mol.GetSubstructMatches(substruct)] # since it does not include wildcard atom
    ###
    
    # get coordinates of all groups #
    grps = []
    for indList in inds:
        grp = []
        for i in indList:
            #atomSym = mol.GetAtomWithIdx(i).GetSymbol()
            #coord = [atomSym] + [pos for pos in mol.GetConformer().GetAtomPosition(i)]
            #grp.append(coord)
            grp.append(coords[i])
        grps.append(grp)
    ###
    # returns coordinates, as well as coordinates of all groups matching the reference molecule
    return coords, grps

# look at these to see what a group looks like #
testGrps1 = getCoordsAndGrps('50_82_84_opt.xyz', dmaSMARTS) # test for TMA groups
testGrps2 =  getCoordsAndGrps('50_86_opt.xyz', molfromXYZ = CRABxyz)
print(testGrps1[1][0], testGrps1[1][0])
#

[['O', 6.724684896, 2.0196819, 14.201645346], ['Al', 5.01431264, 2.28555541, 14.596898568], ['C', 4.092293334, 0.554781065, 14.76921319], ['H', 4.274915013, 0.159747916, 15.783839598], ['H', 4.464458921, -0.182816924, 14.038249601], ['H', 3.001218341, 0.593287069, 14.649877712], ['C', 4.550790837, 3.740818139, 15.849574918], ['H', 5.391328494, 4.079574269, 16.48019175], ['H', 3.796928799, 3.350207813, 16.552845007], ['H', 4.082396605, 4.629213127, 15.39178543]] [['O', 6.724684896, 2.0196819, 14.201645346], ['Al', 5.01431264, 2.28555541, 14.596898568], ['C', 4.092293334, 0.554781065, 14.76921319], ['H', 4.274915013, 0.159747916, 15.783839598], ['H', 4.464458921, -0.182816924, 14.038249601], ['H', 3.001218341, 0.593287069, 14.649877712], ['C', 4.550790837, 3.740818139, 15.849574918], ['H', 5.391328494, 4.079574269, 16.48019175], ['H', 3.796928799, 3.350207813, 16.552845007], ['H', 4.082396605, 4.629213127, 15.39178543]]


Function that takes in a group (as generated above) and rotates it according to some rules. Basically, I'd like to inscribe one of the atoms on a half circle on the positive z-axis (Add how you do this), and rotate it along with its connecting atoms somewhere on that surface. Then, I do another rotation, this time around the bond axis. 

In [4]:
def quatRot(coords):
    names = [coord[0] for coord in coords]
    coords = np.array([coord[1:] for coord in coords], dtype = float)
    
    # make random unit vector in xy plane to rotate around (uniform distribution) #
    randVec = np.array([rand.uniform(-1, 1), rand.uniform(-1, 1), 0])
    randVec = randVec / np.linalg.norm(randVec)
    a1 = rand.uniform(-160, 160) # rotation angle
    q1 = Quaternion(axis = randVec, degrees = a1) # choose rotation around vector in xy plane
    ###
    
    # do another rotation around bond axis #
    bndAxs = coords[1] - coords[0] # bond axis vector
    bndAxs = bndAxs / np.linalg.norm(bndAxs) # normalizing bond axis vector
    a2 = rand.uniform(0, 360)
    q2 = Quaternion(axis = bndAxs, degrees = a2)
    ###
    
    qF = q1 * q2 # composite rotation from both
    
    newCoords = [coord - coords[0] for coord in coords] # center coords at origin
    newCoords = [list(coords[0] + qF.rotate(coord)) for coord in newCoords] # apply rotation
    
    for n in range(len(newCoords)):
        newCoords[n] = [names[n]] + newCoords[n] # return rotated coordinates to input form
        
    return newCoords

1st function (genius coding) gives me VdW radii of the atoms I've been working with. These are used as a rudimentary way of checking for sterics in the rotated conformers in the 2nd function- if the distance between two atoms is less than the sum of their VdW radii, then the rotation is rejected and it has to try again.

In [5]:
def getVdW(name):
    if name == 'Si':
        return 2.10
    if name == 'O':
        return 1.52
    if name == 'C':
        return 1.70
    if name == 'N':
        return 1.55
    return 1.10
    

def distChk(grp, modCoords): # takes rotated grp and whole molecule without grp
    for atom1 in grp[1:]:
        for atom2 in modCoords:
            name1, name2 = atom1[0], atom2[0]
            vdWradiiSum = getVdW(name1) + getVdW(name2)
            
            atomDists = np.linalg.norm(np.array(atom2[1:]) - np.array(atom1[1:]))
            
            if atomDists < vdWradiiSum:
                return False
    return True

The "master" function that rotates groups and outputs successfully rotated geometries as .xyz files. 

In [8]:
def rotGrps(coordFile, nRots, foldername, SMARTSmol = None, XYZmol = None): # takes path of the xyz file of whole structure, smarts, # rotations
    
    # make folder for rotations if there isn't one, removes all previous geoms otherwise #
    if not os.path.exists(foldername):
        os.mkdir(foldername) # make folder for storing rotations
    os.system('rm ' + foldername + '/*')
    ###
    
    # get coords and groups #
    coords, grps = getCoordsAndGrps(coordFile, SMARTS = SMARTSmol, molfromXYZ = XYZmol)
    ###
    
    # function for rotations #
    n = 0 
    while n < nRots:
        grp = grps[rand.randint(0, len(grps) - 1)] # choose random adsorbate group on substrate
        coords_nogrp = [coord for coord in coords if coord not in grp] # get all atoms other than adsorbate
        newGrp = quatRot(grp) # rotate adsorbate
        # newGrp = quatRot2(grp)
        if distChk(newGrp, coords_nogrp): # if sterics are ok, write geom to file and make it new starting geom
            coords = coords_nogrp + newGrp # add rotated grp back to molecule make this new starting coords
            fname = foldername + '/rot_%s.xyz' % str(n + 1)
            with open(fname, 'w') as file: # write rotated coordinates to individual file in folder
                file.write(str(len(coords)) + '\n\n')
                for coord in coords:
                    coord = [str(c) for c in coord]
                    coord = '     '.join(coord) + '\n'
                    file.write(coord)
                    
            grps = [g for g in grps if g != grp ] + [newGrp] # gets all new groups
            n += 1 # loop only moves forward once we get a valid rotation
            
    os.system('cat ' + foldername + '/* > ' + foldername + '/allRots.xyz') # write geoms to trajectory file
    ###        

rotGrps('50_82_84_opt.xyz', 100, 'test', SMARTSmol = dmaSMARTS)

Comments/Concerns:

-I really didn't want to use lists of lists for the groups of atomic coordinates- but I couldn't think of an easier way. Any suggestions?

-The trajectory file I included with this shows the script in action. The atom positions (like in the file text itself) don't stay the same, which glitches out VMD if you try to load it in normally- use the topo readvarxyz command in VMD to see it instead.

-A bigger adsorbate relevant to my research does not work here, seemingly b/c it can't pass the distChk function (basically bad sterics). I could gradually ease the minimum distance requirements, decrease the range of rotations, or make a separate rotation function that just does the bond axis rotation,  but I'm not sure if these are good solutions.

Also, let me know of any coding faux pas I committed, or anything making my code harder to read.

In [ ]:
def quatRot2(coords):
    names = [coord[0] for coord in coords]
    coords = np.array([coord[1:] for coord in coords], dtype = float)
    
    bndAxs = coords[1] - coords[0] # bond axis vector
    bndAxs = bndAxs / np.linalg.norm(bndAxs) # normalizing bond axis vector
    a2 = rand.uniform(0, 360)
    q2 = Quaternion(axis = bndAxs, degrees = a2)
    
    qF = q2 # composite rotation from both
    
    newCoords = [coord - coords[0] for coord in coords] # center coords at origin
    newCoords = [list(coords[0] + qF.rotate(coord)) for coord in newCoords]
    
    for n in range(len(newCoords)):
        newCoords[n] = [names[n]] + newCoords[n]

    return newCoords